Ex.1. kNN own implementation

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics
from six import StringIO  
import pydot 

In [ ]:
db_iris = datasets.load_iris()
X,y=db_iris.data,db_iris.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True,random_state=14)

X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
#computing the Euklides distance

def distance(X_train,X_test_point):
    euclides_distances=[]
    for row in range(X_train.shape[0]):
        curr_distance=0 #current distance
        for column in range(X_train.shape[1]):
            current_point=X_train[row][column]
            curr_distance=np.sqrt(np.sum((current_point-X_test_point[column])**2)) 
        euclides_distances.append(curr_distance)

    distances= pd.DataFrame(data=euclides_distances,columns=['distance'])
    return distances
# nearest neighbours
def NN(distance_pt,k):  
    nearest=distance_pt.sort_values(by=['distance'],axis=0)
    return nearest[:k] #take the only first k neighbours

def pred(nearest,y_train):
    counter = Counter(y_train[nearest.index])
    return counter.most_common()[0][0]

def kNN(X_train,y_train,X_test,k):
    y_pred=[]

    for point in X_test:
        distance_pt=distance(X_train,point)
        nearest=NN(distance_pt,k)
        y_pred_p=pred(nearest,y_train)
        y_pred.append(y_pred_p)

    return y_pred



In [ ]:
from sklearn.preprocessing import Normalizer,StandardScaler

k=3
norm= Normalizer().fit(X_train)
norm_X_train=norm.transform(X_train)
norm_X_test=norm.transform(X_test)
y_pred=kNN(norm_X_train,y_train,norm_X_test,k)
print(accuracy_score(y_test,y_pred))

In [ ]:
k_lst=[]
score_lst=[]
for k in range(1,X_test.shape[0]):
    k_lst.append(k)
    y_pred=kNN(norm_X_train,y_train,norm_X_test,k)
    score_lst.append(accuracy_score(y_test,y_pred))

plt.scatter(k_lst,score_lst,c='r')

Ex.2. kNN from sklearn

In [ ]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
k_lst=[]
score_lst=[]
for k in range(1,X_test.shape[0]):
    k_lst.append(k)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred=knn.predict(X_test)
    score_lst.append(accuracy_score(y_test,y_pred))

plt.scatter(k_lst,score_lst,c='r')


Ex.3 Decision trees

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn import tree
from scipy.io import loadmat
from tensorflow import keras

(x_train, y_train), (x_test, y_test)=keras.datasets.mnist.load_data()

#Ex.1.Divide the data into parameters X and answer y:
 
X = np.concatenate((x_train,x_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)
# Standardization
for i in range(X.shape[0]):
    X[i,:] = X[i,:]/np.std(X[i,:])
 
# Convert digit 10 -> 0 (error in the dataset)    
y[np.where(y==10)]=0
 
# the height and width of the picture with the number
h = 28
w = 28
 
# Ex 2. Please display the number of digits and the number of pixels per image
X.shape


Ex. 3. Please display sample digits from the dataset (functionplot_mnist).


In [ ]:
def plot_mnist(images, titles, h, w, n_row=3, n_col=4):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.05)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)).T, cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

plot_mnist(X,str(y),X.shape[1],X.shape[2])

Ex. 4. Please divide the dataset into learner (70%) and training.

In [ ]:
X=X.reshape(X.shape[0],784)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=77)

Ex. 5. Create an instance of the classifier, then train and predict for test data.
Tree parameters:
DEPTH = 10

In [ ]:
DEPTH=10
clf = tree.DecisionTreeClassifier(max_depth=DEPTH)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Ex 6. Please provide F1 result, confusion matrix and classification report.

In [ ]:
print(f"f1: {sklearn.metrics.f1_score(y_test,y_pred,average='micro')}")

conf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm_display = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix = conf_matrix)
cm_display.plot()
plt.show()

In [ ]:
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write("data.pdf")